# Homework 4

# Задача №1 - Хвост кита

Не так давно на kaggle было соревнование по классификации горбатых китов по снимках их хвостов. Одним из приемов для улучшения работы нейронной сети было выделение маски хвоста. Затем по этой маске вырезался хвост на исходном изображении, нормировался, отображался на черный фон и подавался нейронной сети на вход. 

Ваша задача - написать программу, которая совершит предобработку картинки, описанную выше. Т.е. найти маску хвоста, по маске вырезать из исходного изображения хвост, нормировать изображение хвоста так, чтобы оно всегда имело один заданный размер и равномерно занимало место, то есть, если хвост был повернут, его нужно выпрямить.

Изображения хвостов находятся в папке whale_tail/images к этому занятию. Разметка картинок (маски хвостов) лежат в папке whale_tail/ground_truth. После выделения хвоста посчитайте IoU для каждой картинки (ипсользуя маски из whale_tail/ground_truth)

<table><tr>
	<td> <img src="whale_tail/images/test_image_01.jpg" alt="Drawing" style="width: 400px;"/> 
	<td> <img src="whale_tail/ground_truth/test_image_01.png" alt="Drawing" style="width: 400px;"/> </td>
  
</tr></table>

# IoU 
 Intercetion over union - число от 0 до 1, показывающее, насколько у двух объектов (эталонного (ground true) и текущего) совпадает внутренний “объем”.
 
<table><tr>
	<td> <img src="img/iou.png" alt="Drawing" style="width: 400px;"/> 
  
</tr></table>

In [ ]:
import os

import numpy as np

import cv2 as cv

import skimage

In [6]:
def readImagesFromDirectory(path : str, imread_mode=cv.IMREAD_COLOR) -> list[np.ndarray]:
	images=[]
	
	for file_name in os.listdir(path):
		file_name_full=f"{path}/{file_name}"
		if(os.path.isfile(file_name_full)):
			try:
				if(file_name_full.endswith(('.png', '.jpg', '.jpeg', '.bmp'))):
					images.append(cv.imread(file_name_full, imread_mode))
			except Exception as e:
				print(e)
				pass
			
	return images

In [ ]:
def getTailMask(image : np.ndarray, target_size : tuple[int, int]=(224, 224) ):
	image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
	image_gray_blured = cv.GaussianBlur(cv.cvtColor(image, cv.COLOR_RGB2GRAY), (15, 15), 0)
	contours, _ = cv.findContours(cv.Laplacian(image_gray_blured, cv.THRESH_BINARY, scale=1, ksize=5), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

	mask = np.zeros_like(image_gray_blured)
	contour_largest = max(contours, key=cv.contourArea)
	cv.drawContours(mask, [contour_largest], -1, 255, -1)

	image_result = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
	for i in range(0, image_result.shape[0]):
		for j in range(0, image_result.shape[1]):
			if mask[i,j]==255:
				image_result[i,j]=(126, 95, 223)

	return image_result


In [16]:
def getIOUCoefficient(image1, image2):
	intersection = np.logical_and(image1, image2).sum()
	union = np.logical_or(image1, image2).sum()
	
	if union == 0:
		return 0.0
	
	return intersection / union

In [24]:
path_images_base="lab4/whale_tail/images"
path_images_test="lab4/whale_tail/ground_truth"
images_base=readImagesFromDirectory(path_images_base)
images_masks=readImagesFromDirectory(path_images_test)

In [41]:
i=0
for image in images_base:
	image_processed=cv.cvtColor(getTailMask(image), cv.COLOR_BGR2RGB)
	print(f'{getIOUCoefficient(cv.cvtColor(images_masks[i], cv.COLOR_RGB2BGR), image_processed)}')
	cv.imshow("Base", image)
	cv.imshow("Processed", image_processed)
	cv.waitKey()
	cv.destroyAllWindows()
	i+=1

0.9333710548650906
0.9486940765434323
0.18661734892871074
0.12049311352590703
0.8621588128753456
0.4360679620490594


# Задача №2 - Найти шаблон на изображении с помощью HOG и кросс-корреляции

Ваша задача - написать программу, которая сможет найти шаблон одного глаза на нескольких изображениях, в том числе затемненных. Использовать нужно HOG представление изображений, поиск шаблона сделать с помощью кросс-корреляции 

Изображение глаз человека и шаблон, который нужно на нем найти, находятся в папке hog

<table><tr>
	<td> <img src="hog/eye.png" alt="Drawing" style="width: 400px;"/> 
</tr></table>	 

<table><tr>
	<td> <img src="hog/eyes.jpg" alt="Drawing" style="width: 400px;"/> </td>
	  <td> <img src="hog/eyes_dark.jpg" alt="Drawing" style="width: 400px;"/> </td>
	  <td> <img src="hog/woman.jpg" alt="Drawing" style="width: 200px;"/> </td>

</tr></table>

In [ ]:
def getHogData(image):
	features, hog_image = skimage.feature.hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=-1)

	return features, hog_image

In [ ]:
def findTemplateHog(image, image_template):
	_, image_template_hog = getHogData(image_template)
	_, image_hog = getHogData(image)

	match_result= cv.matchTemplate(image_template_hog.astype(np.float32), image_hog.astype(np.float32), cv.TM_CCOEFF_NORMED)
	value_min, value_max, location_min, location_max = cv.minMaxLoc(match_result)

	return location_max, value_max

In [23]:
path_images_hog="lab4/hog"
images=readImagesFromDirectory(path_images_hog)
image_eye=images.pop(0)

In [27]:
for image in images:
	image_copy=image.copy()
	location, confidence=findTemplateHog(image_copy, image_eye)
	print(f"Confidence: {confidence}")
	cv.rectangle(image_copy, location, (location[0] + image_eye.shape[1], location[1] + image_eye.shape[0]), (255, 0, 0), 2)
	cv.imshow("Correlated", image_copy)
	cv.waitKey()
	cv.destroyAllWindows()

Confidence: 0.9385290145874023
Confidence: 0.9132232069969177
Confidence: 0.5656275153160095
